## Complexity of Rendering
- Geometry
- Shading
- Visibility

## Rendering
**Rendering** refers to the entire process that produces color values for pixels. 
- Compute what's visible. 
- Compute what color it is:
    - Determined by interaction of light and matter
    
### $\text{Rendering} = \text{Scene to image} = \text{Pinhole camera}$
![scene_to_image](images/scene_to_image.png)

### Pinhole camera
- Box with a tiny hole
- Inverted image
- Similar triangles
- **Perfect image if hole is infinitely small**
- Pure geometric optics
- **No depth of field issue**(everything in focus)
![pinhole](images/pinhole.jpg)

## Shading = What surfaces look like
- Surface/Scene properties:
    - Surface normal
    - Direction to light
    - Viewpoint
- Material Properties
    - Diffuse (matte)
    - Specular (shiny)
    - ...
- Light properties
    - Position
    - Intensity
    - ...
- Much more!
![shading_surface](images/shading_surface.png)

### Image synthesis is radiative transport
$$
\begin{aligned} \omega \cdot \nabla_{x} L(x, \omega)=& \epsilon(x, \omega) \\ &-\sigma_{t}(x) L(x, \omega) \\ &+\sigma_{s}(x) \int_{4 \pi} p\left(x, \omega, \omega^{\prime}\right) L\left(x, \omega^{\prime}\right) \mathrm{d} \omega^{\prime} \end{aligned}
$$

### Global Illumination
![illumination](images/illumination.png)

### Ray casting vs. Rasterization
![ray_casting_rasterizaion](images/ray_casting_rasterizaion.jpeg)

### Ray casting vs. Ray tracing
![ray_casting_tracing](images/ray_casting_tracing.png)

### Ray representation
- Two vectors: *Origin* and *Direction*
- Parametric line: $P(t) = \text{origin} + t \times \text{direction}$
    - In words, "Find smallest $t>0$ such that $P(t)$ lies on a surface in the scene."
    
![ray_representation](images/ray_representation.png)

### Ray tracing
- Advantages:
    - Generality: can render anythinig that can be intersected with a ray
    - **Easily allows recursion (shadows, reflections, etc.)**
- Disadvantages:
    - Harder to implement in hardware(less computation coherence, must fit entire scene in memory, worse memory behavior)

### Camera description
- Eye point $e$ (center)
- Orthobasis $u, v, w$ (horizontal, up, direction)

Different coordinate systems are used in a pipeline:
1. Object coordinates
2. World coordinates
3. View coordinates
4. Image coordinates
![view_frustum](images/view_frustum.jpeg)

### Image coordinates
Convenient to define "Normalized Image Coordinates" such that $x, y$ coordinates run from -1 to 1 regardless of the dimensions and aspect ratio of the image rectangle.

![nic](images/nic.jpeg)

### Ray generation in 2D
![ray_generation_2d](images/ray_generation_2d.jpeg)

### Ray generation in 3D
Similar to 2D. $y$ coordinate is treated just like $x$, except accounting for aspect ratio:
$$
\mathbf{r}=\mathrm{x} \cdot \mathbf{u}+\operatorname{aspect} \cdot \mathrm{y} \cdot \mathbf{v}+\mathrm{d} \cdot \mathbf{w}
$$

### Perspective vs. orthographic camera
Orthographic is parallel projection with neither foreshortening nor vanishing point.
![perspective_orthographic](images/perspective_orthographic.png)


Ray generation in orthographic:
- Origin = $\mathbf{e}+\mathrm{x} \cdot \mathrm{size} \cdot \mathbf{u}+\mathrm{y} \cdot \mathrm{size} \cdot \mathbf{v}$
- Direction is constant: $\mathbf{w}$

![orthographic](images/orthographic.png)

### 3D plane representation
- (Infinite) plane is defined by $\mathrm{P}_{\mathrm{o}}=\left(\mathrm{x}_{0}, \mathrm{y}_{0}, \mathrm{Z}_{0}\right)$ and $\mathrm{n}=(\mathrm{A}, \mathrm{B}, \mathrm{C})$.
- Implicit plane equation: 
$$
\begin{aligned} \mathrm{H}(\mathrm{P}) &=\mathrm{Ax}+\mathrm{By}+\mathrm{Cz}+\mathrm{D}=0 \\ &=\mathrm{n} \cdot \mathrm{P}+\mathrm{D}=0 \end{aligned}
$$

- Point-Plane distance?
    - If $n$ is normalized, distance to plane, $d = \mathrm{H}(\mathrm{P})$
    - $d$ is a **signed distance**!

![3d_plane_representation](images/3d_plane_representation.jpeg)


### Explicit vs. Implicit
- Ray equation is explicit: $\mathrm{P}(\mathrm{t})=\mathrm{R}_{\mathrm{o}}+\mathrm{t} \times \mathrm{R}_{\mathrm{d}}$
    - Parametric
    - Generates points
    - Hard to verify that a point is on the ray
    
- Plane equation is implicit: $\mathrm{H}(\mathrm{P})=\mathrm{n} \cdot \mathrm{P}+\mathrm{D}=0$
    - Solution of an equation
    - Does not generate points
    - Verifies that a point is on the plane
    
### Ray-Plane intersection
Intersection means both equations are satisfied. Hence, insert explicit equation of ray into implicit equation of plane, and solve for $t$

$$
\begin{array}{l}{\mathrm{P}(\mathrm{t})=\mathrm{R}_{\mathrm{o}}+\mathrm{t} \times \mathrm{R}_{\mathrm{d}}} \\ {\mathrm{H}(\mathrm{P})=\mathrm{n} \cdot \mathrm{P}+\mathrm{D}=0} \\ {\mathrm{n} \cdot\left(\mathrm{R}_{\mathrm{o}}+\mathrm{t} \times \mathrm{R}_{\mathrm{d}}\right)+\mathrm{D}=0} \\ {\mathrm{t}=-\left(\mathrm{D}+\mathrm{n} \cdot \mathrm{R}_{\mathrm{o}}\right) / \mathrm{n} \cdot \mathrm{R}_{\mathrm{d}}}\end{array}
$$

![ray_plane_intersection](images/ray_plane_intersection.jpeg)

You can verify that,
- intersection is closer than previous: $\mathrm{P}(\mathrm{t})<\mathrm{t}_{\text { current }}$
- it is not out of range(behind eye): $\mathrm{P}(\mathrm{t})>\mathrm{t}_{\mathrm{min}}$

### Normal 
Need surface normal for shading. (e.g. Diffuse: dot product between light direction and normal, clamp to zero). Also, normal is constant over a plane.


### Math digression
- Duality points and planes are "dual" when you use homogenous coordinates
- Point $(x, y, z, 1)$
- Plane $(A, B, C, D)$
- Plane equation $\Rightarrow$ dot product
- You can map planes to points and points to planes in a dual space.
- Lots of cool equivalences
    - intersection of 3 planes define a point
    - 3 points define a plane!


### Ray-Sphere
- Implicit sphere equation: $\mathrm{H}(\mathrm{P})=\|\mathrm{P}\|^{2}-\mathrm{r}^{2}=\mathrm{P} \cdot \mathrm{P}-\mathrm{r}^{2}=0$
- Insert explicit equation of ray into implicit equation of sphere and solve for $t$: 
    - $\mathrm{P}(\mathrm{t})=\mathrm{R}_{\mathrm{o}}+\mathrm{t}^{*} \mathrm{R}_{\mathrm{d}} \quad ; \quad \mathrm{H}(\mathrm{P})=\mathrm{P} \cdot \mathrm{P}-\mathrm{r}^{2}=0$
    - $\left(\mathrm{R}_{\mathrm{o}}+\mathrm{tR}_{\mathrm{d}}\right) \cdot\left(\mathrm{R}_{\mathrm{o}}+\mathrm{tR}_{\mathrm{d}}\right)-\mathrm{r}^{2}=0$
    - $\mathrm{R}_{\mathrm{d}} \cdot \mathrm{R}_{\mathrm{d}} \mathrm{t}^{2}+2 \mathrm{R}_{\mathrm{d}} \cdot \mathrm{R}_{\mathrm{o}} \mathrm{t}+\mathrm{R}_{\mathrm{o}} \cdot \mathrm{R}_{\mathrm{o}}-\mathrm{r}^{2}=0$

![ray-sphere](images/ray-sphere.png)

- It's a quatratic equation: $a t^{2}+b t+c=0$

$$
\begin{array}{l}{\mathrm{a}=\mathrm{R}_{\mathrm{d}} \cdot \mathrm{R}_{\mathrm{d}}} \\ {\mathrm{b}=2 \mathrm{R}_{\mathrm{d}} \cdot \mathrm{R}_{\mathrm{o}}} \\ {\mathrm{c}=\mathrm{R}_{\mathrm{o}} \cdot \mathrm{R}_{\mathrm{o}}-\mathrm{r}^{2}}\end{array}
$$

- discriminant: $d=\sqrt{b^{2}-4 a c}$
- solutions: $t=\frac{-b \pm d}{2 a}$
    - Choose closest positive
    
- Sphere normal is simply $\mathrm{Q} /\|\mathrm{Q}\|$, where $\mathrm{Q}=\mathrm{P}(\mathrm{t})$ is an intersection point (for spheres centered at origin).


![sphere_normal](images/sphere_normal.png)

### 
![](images/.png)



### 
![](images/.png)




### 

### 

### 

### 

### 

### 

### 

### 